In [2]:
%matplotlib inline

In [3]:
import tensorflow as tf
import data_provider
import cacac_train
import models
import numpy as np
import matplotlib.pyplot as plt

from menpo.visualize import print_dynamic
slim = tf.contrib.slim

In [4]:
nogpu_config = tf.ConfigProto(
    # Do not use a GPU device
    device_count = {'GPU': 1}
)

sess = tf.Session(config=nogpu_config)

In [5]:
sess = tf.Session()

In [6]:
audio, ground_truth = data_provider.get_split('/vol/atlas/homes/gt108/db/CACAC/tf_records', 'test', batch_size=1)

In [7]:
tf.train.start_queue_runners(sess=sess)

[<Thread(Thread-4, started daemon 140407675479808)>,
 <Thread(Thread-5, started daemon 140407926220544)>,
 <Thread(Thread-6, started daemon 140405724980992)>,
 <Thread(Thread-7, started daemon 140405716588288)>,
 <Thread(Thread-8, started daemon 140404250322688)>,
 <Thread(Thread-9, started daemon 140404241929984)>]

In [8]:
with slim.arg_scope([slim.batch_norm, slim.layers.dropout],
                    is_training=False):
    prediction = models.get_model('audio')(audio)

In [9]:
variables_to_restore = slim.get_variables_to_restore()
saver = tf.train.Saver(variables_to_restore)
model_path = slim.evaluation.tf_saver.get_checkpoint_state('ckpt/train').model_checkpoint_path
saver.restore(sess, model_path)
print(model_path)

ckpt/train/model.ckpt-45002


In [11]:
preds = []
gts = []

num_samples = 3594

for i in (range(num_samples)):
    a, b = sess.run([tf.argmax(prediction, 1), tf.argmax(ground_truth, 1)])
    preds.append(a[0])
    gts.append(b[0])
    
    print_dynamic("{}/{}, ac: {:.2f}%, rnd: {:.2f}%".format(
            i, num_samples, np.mean(np.array(preds) == np.array(gts)), np.array(gts).mean()))

3593/3594, ac: 0.60%, rnd: 0.60%                                                